In [1]:
import pandas as pd
from utils import heuristic_similarity
import ast

In [2]:
df = pd.read_csv('dataset/movies_all_data2.csv', encoding='utf-8')
df = df.dropna()
df.shape

(20760, 18)

In [3]:
df['genres'] = df['genres'].apply(ast.literal_eval)
df['tags'] = df['tags'].apply(ast.literal_eval)
df['overview_keywords'] = df['overview'].apply(str)
df['characters'] = df['characters'].apply(ast.literal_eval)
df['directors'] = df['directors'].apply(ast.literal_eval)
df['actors'] = df['actors'].apply(ast.literal_eval)


In [4]:
df.sort_values(by='vote_count', ascending=False).head(20)

,adult,id,original_language,original_title,overview,popularity,runtime,tagline,title,vote_average,vote_count,genres,release_year,overview_keywords,tags,directors,characters,actors
9800,False,27205,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.108149,148.0,Your mind is the scene of the crime.,Inception,8.1,14075.0,"[Action, Thriller, Science Fiction, Mystery, A...",2010.0,"Cobb, a skilled thief who commits corporate es...","[loss of lover, dream, kidnapping, sleep, subc...",[Christopher Nolan],"[Dom Cobb, Arthur, Ariadne]","[Leonardo DiCaprio, Joseph Gordon-Levitt, Elle..."
8335,False,155,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167259,152.0,Why So Serious?,The Dark Knight,8.3,12269.0,"[Drama, Action, Crime, Thriller]",2008.0,Batman raises the stakes in his war on crime. ...,"[dc comics, crime fighter, secret identity, sc...","[Christopher Nolan, Steve Gehrke, Nilo Otero]","[Bruce Wayne / Batman, Alfred Pennyworth, Joker]","[Christian Bale, Michael Caine, Heath Ledger]"
9364,False,19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.070892,162.0,Enter the World of Pandora.,Avatar,7.2,12114.0,"[Action, Adventure, Fantasy, Science Fiction]",2009.0,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...",[James Cameron],"[Jake Sully, Princess Neytiri, Dr. Grace Augus...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
10825,False,24428,en,The Avengers,When an unexpected enemy emerges and threatens...,89.887648,143.0,Some assembly required.,The Avengers,7.4,12000.0,"[Science Fiction, Action, Adventure]",2012.0,When an unexpected enemy emerges and threatens...,"[new york, shield, marvel comic, superhero, ba...","[Joss Whedon, Trudy Ramirez]","[Tony Stark / Iron Man, Steve Rogers / Captain...","[Robert Downey Jr., Chris Evans, Mark Ruffalo]"
14611,False,293660,en,Deadpool,Deadpool tells the origin story of former Spec...,187.860492,108.0,Witness the beginning of a happy ending,Deadpool,7.4,11444.0,"[Action, Adventure, Comedy]",2016.0,Deadpool tells the origin story of former Spec...,"[anti hero, mercenary, marvel comic, superhero...",[Tim Miller],"[Wade Wilson / Deadpool, Vanessa Carlysle / Co...","[Ryan Reynolds, Morena Baccarin, Ed Skrein]"
13039,False,157336,en,Interstellar,Interstellar chronicles the adventures of a gr...,32.213481,169.0,Mankind was born on Earth. It was never meant ...,Interstellar,8.1,11187.0,"[Adventure, Drama, Science Fiction]",2014.0,Interstellar chronicles the adventures of a gr...,"[saving the world, artificial intelligence, fa...",[Christopher Nolan],"[Joseph Cooper, Murph Cooper, Dr. Amelia Brand]","[Matthew McConaughey, Jessica Chastain, Anne H..."
11778,False,68718,en,Django Unchained,"With the help of a German bounty hunter, a fre...",19.785025,165.0,"Life, liberty and the pursuit of vengeance.",Django Unchained,7.8,10297.0,"[Drama, Western]",2012.0,"With the help of a German bounty hunter, a fre...","[bounty hunter, hero, plantation, society, fri...","[Quentin Tarantino, Martin Kitrosser, William ...","[Django, Dr. King Schultz, Calvin Candie]","[Jamie Foxx, Christoph Waltz, Leonardo DiCaprio]"
13444,False,118340,en,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",53.291601,121.0,All heroes start somewhere.,Guardians of the Galaxy,7.9,10014.0,"[Action, Science Fiction, Adventure]",2014.0,"Light years from Earth, 26 years after being a...","[marvel comic, spaceship, space, outer space, ...",[James Gunn],"[Peter Quill / Star-Lord, Gamora, Drax the Des...","[Chris Pratt, Zoe Saldana, Dave Bautista]"
2192,False,550,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,63.869599,139.0,Mischief. Mayhem. Soap.,Fight Club,8.3,9678.0,[Drama],1999.0,A ticking-time-bomb insomniac and a slippery s...,"[support group, dual identity, nihilism, rage ...","[David Fincher, Dina Waxman]","[The Narrator, Tyler Durden, Robert 'Bob' Paul...","[Edward Norton, Brad 

In [5]:
heuristic_similarity(df, 49051, 5)

search: The Hobbit: An Unexpected Journey


,id,title,genres,overview_keywords,tags,characters,directors,actors
0,122917,The Hobbit: The Battle of the Five Armies,"[Action, Adventure, Fantasy]",Immediately after the events of The Desolation...,"[corruption, elves, dwarves, orcs, middle-eart...","[Bilbo Baggins, Gandalf, Thorin Oakenshield]",[Peter Jackson],"[Martin Freeman, Ian McKellen, Richard Armitage]"
1,57158,The Hobbit: The Desolation of Smaug,"[Adventure, Fantasy]","The Dwarves, Bilbo and Gandalf have successful...","[elves, dwarves, orcs, hobbit, dragon, wizard,...","[Bilbo Baggins, Gandalf, Thorin Oakenshield]","[Peter Jackson, Carolynne Cunningham, Veroniqu...","[Martin Freeman, Ian McKellen, Richard Armitage]"
2,17632,The Hunt for Gollum,"[Action, Adventure, Fantasy]",A British fan film based on the appendices of ...,"[middle-earth (tolkien), the lord of the rings]","[Aragorn, Gandalf, Arithir]",[Chris Bouchard],"[Adrian Webster, Patrick O'Connor, Arin Alldri..."
3,120,The Lord of the Rings: The Fellowship of the Ring,"[Adventure, Fantasy, Action]","Young hobbit Frodo Baggins, after inheriting a...","[elves, dwarves, orcs, middle-earth (tolkien),...","[Frodo Baggins, Gandalf the Grey, Galadriel]",[Peter Jackson],"[Elijah Wood, Ian McKellen, Cate Blanchett]"
4,121,The Lord of the Rings: The Two Towers,"[Adventure, Fantasy, Action]",Frodo and Sam are trekking to Mordor to destro...,"[elves, orcs, middle-earth (tolkien), hobbit, ...","[Frodo Baggins, Gandalf the White, Aragorn]",[Peter Jackson],"[Elijah Wood, Ian McKellen, Viggo Mortensen]"


In [6]:
heuristic_similarity(df, "godfather", 5)

Found matching titles:
          id                    title
614      238            The Godfather
871      240   The Godfather: Part II
1457     242  The Godfather: Part III
5715   13398         Tokyo Godfathers
6099   29244             3 Godfathers
12753  77403          Disco Godfather


In [7]:
heuristic_similarity(df, "The Godfather", 5)

search: The Godfather


,id,title,genres,overview_keywords,tags,characters,directors,actors
0,240,The Godfather: Part II,"[Drama, Crime]",In the continuing saga of the Corleone crime f...,"[italo-american, cuba, vororte, melancholy, pr...","[Don Michael Corleone, Tom Hagen, Kay Corleone]","[Francis Ford Coppola, John Franco, Newt Arnol...","[Al Pacino, Robert Duvall, Diane Keaton]"
1,44800,The Gambler,"[Drama, Crime]",New York City English professor Axel Freed out...,"[gambling, professor, mafia, money, debt]","[Axel Freed, Hips, Billie]",[Karel Reisz],"[James Caan, Paul Sorvino, Lauren Hutton]"
2,242,The Godfather: Part III,"[Crime, Drama, Thriller]",In the midst of trying to legitimize his busin...,"[italy, christianity, new york, assassination,...","[Don Michael Corleone, Kay Adams Michelson, Do...",[Francis Ford Coppola],"[Al Pacino, Diane Keaton, Andy García]"
3,94741,The Don Is Dead,"[Crime, Drama]",After his mistress is murdered a Mafia leader ...,[mafia],"[Don Angelo, Tony Fargo, Frank]",[Richard Fleischer],"[Anthony Quinn, Frederic Forrest, Robert Forster]"
4,26762,The Outfit,"[Crime, Drama]",A two-bit criminal takes on the Mafia to aveng...,"[revenge, suspense, mafia, ex-con]","[Macklin, Bett Harrow, Cody]","[John Flynn, William McGarry]","[Robert Duvall, Karen Black, Joe Don Baker]"


In [8]:
heuristic_similarity(df, 68721, 10)

search: Iron Man 3


,id,title,genres,overview_keywords,tags,characters,directors,actors
0,10138,Iron Man 2,"[Adventure, Action, Science Fiction]",With the world now aware of his dual life as t...,"[malibu, marvel comic, superhero, based on com...","[Tony Stark / Iron Man, Virginia ""Pepper"" Pott...",[Jon Favreau],"[Robert Downey Jr., Gwyneth Paltrow, Don Cheadle]"
1,1726,Iron Man,"[Action, Science Fiction, Adventure]","After being held captive in an Afghan cave, bi...","[middle east, arms dealer, malibu, marvel comi...","[Tony Stark / Iron Man, Lt. Col. James ""Rhodey...","[Jon Favreau, Cristina Weigmann]","[Robert Downey Jr., Terrence Howard, Jeff Brid..."
2,99861,Avengers: Age of Ultron,"[Action, Adventure, Science Fiction]",When Tony Stark tries to jumpstart a dormant p...,"[marvel comic, sequel, superhero, based on com...","[Tony Stark / Iron Man, Thor Odinson, Bruce Ba...",[Joss Whedon],"[Robert Downey Jr., Chris Hemsworth, Mark Ruff..."
3,271110,Captain America: Civil War,"[Adventure, Action, Science Fiction]","Following the events of Age of Ultron, the col...","[civil war, war, marvel comic, sequel, superhe...","[Steve Rogers / Captain America, Tony Stark / ...","[Anthony Russo, Joe Russo, Joe Russo]","[Chris Evans, Robert Downey Jr., Scarlett Joha..."
4,24428,The Avengers,"[Science Fiction, Action, Adventure]",When an unexpected enemy emerges and threatens...,"[new york, shield, marvel comic, superhero, ba...","[Tony Stark / Iron Man, Steve Rogers / Captain...","[Joss Whedon, Trudy Ramirez]","[Robert Downey Jr., Chris Evans, Mark Ruffalo]"
5,102899,Ant-Man,"[Science Fiction, Action, Adventure]",Armed with the astonishing ability to shrink i...,"[marvel comic, superhero, based on comic, afte...","[Scott Lang / Ant-Man, Dr. Hank Pym, Hope van ...","[Peyton Reed, Kerry Lyn McKissick, Lars P. Win...","[Paul Rudd, Michael Douglas, Evangeline Lilly]"
6,100402,Captain America: The Winter Soldier,"[Action, Adventure, Science Fiction]",After the cataclysmic events in New York with ...,"[washington d.c., future, shield, marvel comic...","[Steve Rogers / Captain America, Nick Fury, Na...","[Anthony Russo, Joe Russo]","[Chris Evans, Samuel L. Jackson, Scarlett Joha..."
7,1771,Captain America: The First Avenger,"[Action, Adventure, Science Fiction]","Predominantly set during World War II, Steve R...","[new york, usa, world war ii, nazis, marvel co...","[Steve Rogers / Captain America, Johann Schmid...",[Joe Johnston],"[Chris Evans, Hugo Weaving, Tommy Lee Jones]"
8,36657,X-Men,"[Adventure, Action, Science Fiction]","Two mutants, Rogue and Wolverine, come to a pr...","[mutant, marvel comic, superhero, based on com...","[Charles Xavier / Professor X, Logan / Wolveri...",[Bryan Singer],"[Patrick Stewart, Hugh Jackman, Ian McKellen]"
9,1724,The Incredible Hulk,"[Science Fiction, Action, Adventure]",Scientist Bruce Banner scours the planet for a...,"[new york, rio de janeiro, marvel comic, super...","[Bruce Banner / The Hulk, Betty Ross, Emil Blo...","[Louis Leterrier, Louis Leterrier]","[Edward Norton, Liv Tyler, Tim Roth]"


In [9]:
heuristic_similarity(df, "The Lord of the Rings: The Return of the King", 15)

search: The Lord of the Rings: The Return of the King


,id,title,genres,overview_keywords,tags,characters,directors,actors
0,121,The Lord of the Rings: The Two Towers,"[Adventure, Fantasy, Action]",Frodo and Sam are trekking to Mordor to destro...,"[elves, orcs, middle-earth (tolkien), hobbit, ...","[Frodo Baggins, Gandalf the White, Aragorn]",[Peter Jackson],"[Elijah Wood, Ian McKellen, Viggo Mortensen]"
1,120,The Lord of the Rings: The Fellowship of the Ring,"[Adventure, Fantasy, Action]","Young hobbit Frodo Baggins, after inheriting a...","[elves, dwarves, orcs, middle-earth (tolkien),...","[Frodo Baggins, Gandalf the Grey, Galadriel]",[Peter Jackson],"[Elijah Wood, Ian McKellen, Cate Blanchett]"
2,122917,The Hobbit: The Battle of the Five Armies,"[Action, Adventure, Fantasy]",Immediately after the events of The Desolation...,"[corruption, elves, dwarves, orcs, middle-eart...","[Bilbo Baggins, Gandalf, Thorin Oakenshield]",[Peter Jackson],"[Martin Freeman, Ian McKellen, Richard Armitage]"
3,17632,The Hunt for Gollum,"[Action, Adventure, Fantasy]",A British fan film based on the appendices of ...,"[middle-earth (tolkien), the lord of the rings]","[Aragorn, Gandalf, Arithir]",[Chris Bouchard],"[Adrian Webster, Patrick O'Connor, Arin Alldri..."
4,49051,The Hobbit: An Unexpected Journey,"[Adventure, Fantasy, Action]","Bilbo Baggins, a hobbit enjoying his quiet lif...","[riddle, elves, dwarves, orcs, middle-earth (t...","[Gandalf, Bilbo, Thorin]","[Peter Jackson, Victoria Sullivan]","[Ian McKellen, Martin Freeman, Richard Armitage]"
5,68735,Warcraft,"[Action, Adventure, Fantasy]",The peaceful realm of Azeroth stands on the br...,"[video game, elves, orcs, magic, chase, based ...","[Garona Halforcen, Anduin Lothar, Medivh]","[Duncan Jones, Jessica Clothier]","[Paula Patton, Travis Fimmel, Ben Foster]"
6,59883,Age of the Dragons,"[Adventure, Action, Fantasy]",Age of the Dragons is an adaptation of Herman ...,"[based on novel, obsession, hero, harpoon, rev...","[Ahab, Stubb, Ishmael]",[Ryan Little],"[Danny Glover, Vinnie Jones, Corey Sevier]"
7,44126,Sinbad of the Seven Seas,"[Action, Adventure, Fantasy]",Fantasy tale based on the tale of the legendar...,"[sinbad, wizard, arabian nights, warrior]","[Sinbad, Jaffar, Alì]","[Enzo G. Castellari, Luigi Cozzi, Stefania Gir...","[Lou Ferrigno, John Steiner, Roland Wybenga]"
8,245842,The King's Daughter,"[Fantasy, Action, Adventure]",King Louis XIV's quest for immortality leads h...,"[immortality, mermaid, period drama, illegitim...","[King Louis XIV, Pere La Chaise, Yves De La Cr...","[Sean McNamara, Mathieu Turi]","[Pierce Brosnan, William Hurt, Benjamin Walker]"
9,58,Pirates of the Caribbean: Dead Man's Chest,"[Adventure, Fantasy, Action]",Captain Jack Sparrow works his way out of a bl...,"[witch, fortune teller, bondage, exotic island...","[Captain Jack Sparrow, Will Turner, Elizabeth ...",[Gore Verbinski],"[Johnny Depp, Orlando Bloom, Keira Knightley]"
